# 3. Pre-processing and Training Data Development

* [3 Training Data](#2_Data_training_introduction)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
print("Loaded Libraries")

/opt/homebrew/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Loaded Libraries


In [2]:
# Load the data
products = pd.read_csv("../data/processed/products.csv")
new_categories = pd.read_csv("../data/processed/Sunlight-Categories.csv")["Category"].tolist()

# Convert non-categorical columns to categorical format
products['Brand'] = products['Brand'].astype('category')
products['Color'] = products['Color'].astype('string')
products['Size'] = products['Size'].astype('string')
products['Description'] = products['Category'].astype('string')
products['ParentCategory'] = products['ParentCategory'].astype('category')

# Define numerical and categorical features
numerical_features = ['MSRP']
categorical_features = ['Brand', 'Size', 'Color', 'Description', 'ParentCategory', 'Category']
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21396 entries, 0 to 21395
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Category        21396 non-null  object  
 1   Brand           18956 non-null  category
 2   Description     21396 non-null  string  
 3   Keyword         18956 non-null  object  
 4   UPC             18970 non-null  object  
 5   MSRP            21396 non-null  float64 
 6   Quantity        21396 non-null  int64   
 7   SKU             21396 non-null  object  
 8   Color           15387 non-null  string  
 9   Size            15806 non-null  string  
 10  StyleNumber     7000 non-null   object  
 11  StyleName       8881 non-null   object  
 12  ParentCategory  21396 non-null  category
dtypes: category(2), float64(1), int64(1), object(6), string(3)
memory usage: 1.9+ MB


In [3]:
# Define preprocessing steps
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline
# Now we have a full prediction pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier())])

# Define X and y
X = products.drop(columns=['Category']) 
y = products['Category']
X.head()
y.head()

0                             Clothing - Socks
1                         Ski Hardgoods - Skis
2    Accessories - Summer - Clothing - Gloves 
3                Clothing - Winter - Outerwear
4            Accessories - Summer - Rear Racks
Name: Category, dtype: object

In [4]:
# Rename the column header to 'category'
y = y.to_frame(name='Category')

# Now you can use y.head() to verify the changes
print(y.head())

                                    Category
0                           Clothing - Socks
1                       Ski Hardgoods - Skis
2  Accessories - Summer - Clothing - Gloves 
3              Clothing - Winter - Outerwear
4          Accessories - Summer - Rear Racks


Hi Reviewer! I need help with these last lines of code. It appears that y does not have enough data. I am not sure what is going on and would love some help!

In [10]:

# Split data into training and testing sets (80% training, 20% testing)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

print(y_train, y_test )

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [9]:
# Train the model
pipeline.fit(X_train, y_train)

# Evaluate the model
accuracy = pipeline.score(X_test, y_test)
print("Model Accuracy:", accuracy)


ValueError: A given column is not a column of the dataframe